In [1]:
import sys
from pyscf.lib import logger
from ad_afqmc.lno.afqmc import LNOAFQMC
from ad_afqmc import pyscf_interface
from pyscf import gto, scf, mcscf, ao2mo
import numpy as np
from functools import reduce
_fdot = np.dot
fdot = lambda *args: reduce(_fdot, args)

In [2]:
log = logger.Logger(sys.stdout, 6)

d = 100
basis = 'cc-pvdz'
atom1 = '''
O  -1.551007  -0.114520   0.000000
H  -1.934259   0.762503   0.000000
H  -0.599677   0.040712   0.000000
O   1.350625   0.111469   0.000000
H   1.680398  -0.373741  -0.758561
H   1.680398  -0.373741   0.758561
'''
atom2 = f'''
O  -1.551007  -0.114520   0.000000
H  -1.934259   0.762503   0.000000
H  -0.599677   0.040712   0.000000
O   {1.350625+d}   0.111469   0.000000
H   {1.680398+d}  -0.373741  -0.758561
H   {1.680398+d}  -0.373741   0.758561
'''

mol1 = gto.M(atom=atom1,basis=basis,verbose=4)
mol2 = gto.M(atom=atom2,basis=basis,verbose=4)

mf1 = scf.RHF(mol1).density_fit()
mf1.newton()
mf1.kernel()

mf2 = scf.RHF(mol2).density_fit()
mf2.newton()
mf2.kernel()

frozen = 2
    
## canonical
# mmp1 = mp.MP2(mf1, frozen=frozen)
# mmp1.kernel()[0]

# mmp2 = mp.MP2(mf2, frozen=frozen)
# mmp2.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Thu Feb 13 22:15:57 2025
PySCF version 2.7.0
PySCF path  /home/yichi/research/software/pyscf
GIT HEAD (branch master) a0665c4a7bf54e33f01295b3eea390be7a17d76d

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 20
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O     -1.551007000000  -0.114520000000   0.000000000000 AA   -2.930978447283  -0.216411435785   0.00000

In [ ]:
options = {
    "dt": 0.01,
    "n_exp_terms": 6,
    "n_walkers": 30,
    "n_runs": 100,
    "rlx_steps": 0,
    "prop_steps": 10,
    "seed": 23,
    "walker_type": "rhf",
    "trial": "rhf",
    "use_gpu": False,
    "free_proj": False,
}
from ad_afqmc.corr_sample_test import lnocs
lnocs.run_cs_frags(mf1,mf2,2,options,lno_thresh=1e-4)
# mk_cs_frag(6,mf1,mf2,2,options,chol_cut=1e-6)

lo_type = pm
lo_type = pm
# Preparing LNO-AFQMC calculation
# Generating Cholesky Integrals
# Decomposing ERI with DF
# local active orbitals are [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
# local active space size 13
# chol shape: (232, 169)
# Finished calculating Cholesky integrals

# Size of the correlation space:
# Number of electrons: (4, 4)
# Number of basis functions: 13
# Number of Cholesky vectors: 232

# Preparing LNO-AFQMC calculation
# Generating Cholesky Integrals
# Decomposing ERI with DF
# local active orbitals are [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
# local active space size 13
# chol shape: (232, 169)
# Finished calculating Cholesky integrals

# Size of the correlation space:
# Number of electrons: (4, 4)
# Number of basis functions: 13
# Number of Cholesky vectors: 232

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


In [ ]:
import pickle
import os
def run_lnocs_afqmc(options,files,nproc=None):

    with open("options.bin", "wb") as f:
        pickle.dump(options, f)
    with open("files.bin", "wb") as f:
        pickle.dump(files, f)
    # if script is None:
    path = os.path.abspath(__file__)
    dir_path = os.path.dirname(path)
    script = f"{dir_path}/run_lnocs.py"
    use_gpu = options["use_gpu"]
    if use_gpu:
        # config.afqmc_config["use_gpu"] = True
        gpu_flag = "--use_gpu"
    else: gpu_flag = ""
    if mpi_prefix is None:
        if use_gpu:
            mpi_prefix = ""
        else:
            mpi_prefix = "mpirun "
    if nproc is not None:
        mpi_prefix += f"-np {nproc} "
    os.system(
        f"export OMP_NUM_THREADS=1; export MKL_NUM_THREADS=1; {mpi_prefix} python {script} {gpu_flag} |tee lnocs_afqmc.out"
    )